# Generate morphotope geometry with cluster information

Dissolves enclosed tessellation cells into morphotopes for visualisation purposes and links cluster information we have on buildings to each morphotope. To be displayed on the interactive map.

In [1]:
import os

import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal import graph
from tqdm.auto import tqdm

In [2]:
folder = "/data/uscuni-ulce/processed_data/tessellations/"
clusters_folder = "/data/uscuni-ulce/processed_data/clusters/"
morphotope_folder = "/data/uscuni-ulce/processed_data/morphotope_clusters/"

In [3]:
v = "v10"

In [4]:
# os.mkdir(morphotope_folder)

In [5]:
country = "fr_sp_nl_be"
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + f"{country}_regions_hull.parquet"
)


In [6]:
model_params = '_post_processing_v1'

In [27]:
def generate_morphotope_geometry(region_id):

    # read data
    tess = gpd.read_parquet(f"{folder}tessellation_{region_id}.parquet")
    blg = gpd.read_parquet(f"{clusters_folder}clusters_{region_id}_{v}.pq")
    tess_w_data = tess.join(blg[["morph", "unique_morph", "final", "final_without_noise"]], how="inner")

    morphotopes = tess_w_data.dissolve("unique_morph")
    morphotopes.geometry = morphotopes.buffer(.01)
    
    morphotopes.to_parquet(f"{morphotope_folder}{v}/{region_id}_clusters.pq")

In [29]:
%%time

from joblib import Parallel, delayed
n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(generate_morphotope_geometry)(region_id) for region_id, _ in region_hulls.iterrows()
)

/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 1.29 s, sys: 135 ms, total: 1.42 s
Wall time: 6min 45s
